In [2]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import  matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error

from random import sample
seed_list = list(range(10000))

import warnings
warnings.simplefilter('ignore')
os.chdir('C:\\Users\\yeonjun.in\\Desktop\\연준\\캐글\\제주도\\data\\raw')

TODAY = str(datetime.now().year)+str(datetime.now().month)+str(datetime.now().day)

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv("test.csv")

# sub = pd.read_csv('submission_sample.csv')

os.chdir('C:\\Users\\yeonjun.in\\Desktop\\연준\\캐글\\제주도\\code\\experiment')
experiment_db = pd.read_csv('experiment_DB.csv')

In [27]:
train.columns  = ['id', 'date', 'bus_route_id', 'in_out', 'station_code', 'station_name',
 'latitude', 'longitude', 'ride_6_7', 'ride_7_8', 'ride_8_9',
 'ride_9_10', 'ride_10_11', 'ride_11_12', 'takeoff_6_7', 'takeoff_7_8',
 'takeoff_8_9', 'takeoff_9_10', 'takeoff_10_11', 'takeoff_11_12',
 'ride_18_20']
test.columns = ['id', 'date', 'bus_route_id', 'in_out', 'station_code', 'station_name',
 'latitude', 'longitude', 'ride_6_7', 'ride_7_8', 'ride_8_9',
 'ride_9_10', 'ride_10_11', 'ride_11_12', 'takeoff_6_7', 'takeoff_7_8',
 'takeoff_8_9', 'takeoff_9_10', 'takeoff_10_11', 'takeoff_11_12'] 

In [28]:
train['date'] = pd.to_datetime(train['date'])
train['weekday'] = train['date'].dt.weekday
train = pd.get_dummies(train,columns=['weekday'])
train['weekday_var'] = train['date'].dt.weekday

test['date'] = pd.to_datetime(test['date'])
test['weekday'] = test['date'].dt.weekday
test = pd.get_dummies(test,columns=['weekday'])
test['weekday_var'] = test['date'].dt.weekday

In [29]:
train['in_out'] = train['in_out'].map({'시내':0,'시외':1})
test['in_out'] = test['in_out'].map({'시내':0,'시외':1})

In [30]:
import geopy.distance

coords_jejusi = (33.500770, 126.522761) #제주시의 위도 경도
coords_seoquipo = (33.259429, 126.558217) #서귀포시의 위도 경도


train['dis_jejusi'] = [geopy.distance.vincenty((train['latitude'].iloc[i],train['longitude'].iloc[i]), coords_jejusi).km for i in range(len(train))]
train['dis_seoquipo'] = [geopy.distance.vincenty((train['latitude'].iloc[i],train['longitude'].iloc[i]), coords_seoquipo).km for i in range(len(train))]

test['dis_jejusi'] = [geopy.distance.vincenty((test['latitude'].iloc[i],test['longitude'].iloc[i]), coords_jejusi).km for i in range(len(test))]
test['dis_seoquipo'] = [geopy.distance.vincenty((test['latitude'].iloc[i],test['longitude'].iloc[i]), coords_seoquipo).km for i in range(len(test))]

In [31]:
train['ride_6_12'] = train[['ride_6_7','ride_7_8','ride_8_9','ride_9_10','ride_10_11','ride_11_12']].sum(axis=1)
test['ride_6_12'] = test[['ride_6_7','ride_7_8','ride_8_9','ride_9_10','ride_10_11','ride_11_12']].sum(axis=1)

train['takeoff_6_12'] = train[['takeoff_6_7','takeoff_7_8','takeoff_8_9','takeoff_9_10','takeoff_10_11','takeoff_11_12']].sum(axis=1)
test['takeoff_6_12'] = test[['takeoff_6_7','takeoff_7_8','takeoff_8_9','takeoff_9_10','takeoff_10_11','takeoff_11_12']].sum(axis=1)

In [34]:
os.chdir('C:\\Users\\yeonjun.in\\Desktop\\연준\\캐글\\제주도\\data')

In [35]:
train.to_pickle('fixed1_train.pkl')
test.to_pickle('fixed1_test.pkl')

In [141]:
added = []
input_var=['in_out','latitude','longitude','ride_6_7', 'ride_7_8', 'ride_8_9', 
           'ride_9_10','ride_10_11', 'ride_11_12','ride_6_12',
           'takeoff_6_7', 'takeoff_7_8', 'takeoff_8_9','takeoff_9_10', 
           'takeoff_10_11', 'takeoff_11_12','takeoff_6_12',
           'weekday_0', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4',
           'weekday_5', 'weekday_6', 
           'dis_jejusi', 'dis_seoquipo']
target=['ride_18_20']

In [173]:
holi = ['2019-09-12','2019-09-13','2019-09-14', '2019-10-03','2019-10-09']
wkend = ['2019-09-01','2019-09-07','2019-09-08','2019-09-14','2019-09-15',
         '2019-09-21','2019-09-22','2019-09-28','2019-09-29',
        '2019-10-05','2019-10-06','2019-10-12','2019-10-13']
workday = sorted(list(set(pd.concat([train.date,test.date],axis=0).astype('str').unique()) - set(holi+wkend)))
train['day_type'] =  np.where(train.date.isin(holi),1, 
                            np.where(train.date.isin(wkend),2,3))
test['day_type'] =  np.where(test.date.isin(holi),1, 
                            np.where(test.date.isin(wkend),2,3))



train = pd.get_dummies(train,columns=['day_type'])
test = pd.get_dummies(test,columns=['day_type'])

added + = [a for a in train.columns if 'day_type' in a]

In [142]:
X_train=train[input_var]
y_train=train[target]

X_test=test[input_var]

In [151]:
NFOLDS = 6
random_seed = sample(seed_list,1)

tscv = TimeSeriesSplit(max_train_size=None, n_splits=NFOLDS)
stk = StratifiedKFold(n_splits=NFOLDS,random_state = random_seed[0],shuffle=True)

In [152]:
experiment_info = ''

In [153]:
start = datetime.now()

time = str(start.hour)+'hr'
minute = str(start.minute)+'min'

cv_train = np.zeros(len(y_train))
cv_pred = np.zeros(test.shape[0])
fold_scores = []


for fold_, (tr_index, vl_index) in enumerate(stk.split(X_train,y_train)):
# for fold_, (tr_index, vl_index) in enumerate(tscv.split(train)):    
    print('Fold:', fold_+1)
    tst = X_test[input_var]
    tr, vl = X_train.iloc[tr_index], X_train.iloc[vl_index]
    y_tr, y_vl = y_train.iloc[tr_index], y_train.iloc[vl_index]
    
#     print(tr_index,vl_index)
    
    rf = RandomForestRegressor(random_state=random_seed[0],n_estimators=100,criterion='mse')
    rf.fit(tr,y_tr)
    
    pred = rf.predict(vl)
    
    print(np.sqrt(mean_squared_error(y_vl,pred)))
    cv_train[vl_index] += pred
    cv_pred += rf.predict(tst)
    
    print('-'*40+'\n\n')
    
cv_pred /= NFOLDS

vl_error = np.sqrt(mean_squared_error(np.array(y_train).flatten(),cv_train))

print('cv score:')
print(vl_error)
# print(fold_scores)
# print(best_trees,np.mean(best_trees))

sub['18~20_ride'] = cv_pred

end = datetime.now()

print(end - start)

Fold: 1
3.0578535509735802
----------------------------------------


Fold: 2
3.091997648033784
----------------------------------------


Fold: 3
2.884979108721189
----------------------------------------


Fold: 4
2.968418145559636
----------------------------------------


Fold: 5
2.890774293521982
----------------------------------------


Fold: 6
2.895681225176614
----------------------------------------


cv score:
2.966140303929837
0:02:31.570170


In [133]:
os.chdir('C:\\\Users\\yeonjun.in\\Desktop\\연준\\캐글\\제주도\\submission')
sub['18~20_ride'] = cv_pred
sub.to_csv(TODAY+'_'+time+'_'+minute+'_'+'sub'+'.csv',index=False)

In [118]:
experiment_result = pd.DataFrame({
    'date' : [TODAY],
    'time' : [time],
    'score' : [vl_error],
    'feature_list' : [input_var],
    'model' : 'rf',
    'hyper_param' : 'n_estimatior : 10
    'explanation' : experiment_info
})

In [119]:
os.chdir('C:\\Users\\yeonjun.in\\Desktop\\연준\\캐글\\제주도\\code\\experiment')
pd.concat([experiment_db,experiment_result],axis=0).to_csv('experiment_DB.csv',index=False)